<a href="https://colab.research.google.com/github/KPSawhney/mesop/blob/main/Shopify_AI_Create_Dummy_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shopify AI: Create Dummy Data!

In this colab, we generate some dummy Shopify product data, which we will later feed into our chatbot.

# Install necessary deps & imports

In [ ]:
!pip install google-cloud-bigquery faker

In [140]:
import csv
import io
import random
import time

import google.generativeai as genai

from datetime import date, datetime, timedelta
from faker import Faker
from google.cloud import bigquery
from google.colab import auth, userdata

# Define constants

In [128]:
# Define the schema for the Shopify products table
PRODUCTS_SCHEMA = [
    bigquery.SchemaField("product_id", "INT64"),
    bigquery.SchemaField("product_type", "STRING"),
    bigquery.SchemaField("title", "STRING"),
    bigquery.SchemaField("status", "STRING"),
    bigquery.SchemaField("created_timestamp", "TIMESTAMP"),
    bigquery.SchemaField("collections", "STRING"),
    bigquery.SchemaField("count_variants", "INT64"),
    bigquery.SchemaField("has_product_image", "BOOL"),
    bigquery.SchemaField("total_quantity_sold", "FLOAT64"),
    bigquery.SchemaField("subtotal_sold", "FLOAT64"),
    bigquery.SchemaField("quantity_sold_net_refunds", "FLOAT64"),
    bigquery.SchemaField("subtotal_sold_net_refunds", "FLOAT64"),
    bigquery.SchemaField("product_total_discount", "FLOAT64"),
    bigquery.SchemaField("product_total_tax", "FLOAT64"),
]

COLLECTIONS = [
    'SS','AW','Collaborations'
]

PRODUCT_TYPE = [
    'Trousers',
    'Shirts',
    'Slippers',
    'Top Hats'
]

VENDORS = [
    'eComm',
    'Physical Store'
]

STATUS = 'Active'

PROJECT_ID = 'mesop-431518'

# Authenticate with Google, and setup the Faker for our dummy data

In [129]:
auth.authenticate_user()
fake = Faker()
client = bigquery.Client(project=PROJECT_ID)

# Fn Definition to generate some dummy data!

In [134]:
# Generate dummy data
def generate_dummy_product():
    product_type = random.choice(PRODUCT_TYPE)
    vendor = random.choice(VENDORS)
    collection = random.choice(COLLECTIONS)

    total_quantity_sold = random.randint(1,100)
    price = random.randint(5,100)
    total_quantity_returned = round(total_quantity_sold * random.randint(1,2)/10,0)

    total_sold = round(total_quantity_sold * price, 2)
    total_discount = round(total_sold * random.randint(1, 2)/10, 0)

    subtotal_sold = total_sold - total_discount

    created_ts = fake.date_time_between_dates(
        datetime_start=date(date.today().year, 1, 1),
        datetime_end=datetime.now() - timedelta(days=7)
    ).isoformat()

    net_refunds = round(subtotal_sold / random.randint(1, 10), 0)
    total_discount = round(total_quantity_sold * random.randint(1, 2)/10, 0)

    return {
        "product_id": fake.random_int(min=1, max=1000000),
        "product_type": product_type,
        "title": f'{fake.slug()}: {product_type}',
        "status": STATUS,
        "created_timestamp": created_ts,
        "collections": collection,
        "count_variants": fake.random_int(min=1, max=10),
        "has_product_image": fake.boolean(),
        "total_quantity_sold": total_quantity_sold,
        "subtotal_sold": subtotal_sold,
        "quantity_sold_net_refunds": total_quantity_sold - total_quantity_returned,
        "subtotal_sold_net_refunds": subtotal_sold - (total_quantity_returned * price),
        "product_total_discount": total_discount,
        "product_total_tax": (subtotal_sold - net_refunds) * 0.2,
    }

# Generate the dummy data & upload it to BigQuery

In [135]:
# Generate the dummy data
dummy_data = [generate_dummy_product() for _ in range(1000)]

table_id = "mesop-431518.shopify_ai.shopify_products_dummy"
table = bigquery.Table(table_id, schema=PRODUCTS_SCHEMA)

table = client.create_table(table, exists_ok=True)

errors = client.insert_rows_json(table, dummy_data)

if errors:
    print(f"Encountered errors: {errors}")
else:
    print("Successfully inserted rows into BigQuery table.")

Successfully inserted rows into BigQuery table.


# Get results from BQ table & convert to TSV

In [ ]:
# Define your SQL query to retrieve data
query = f"""
SELECT
  *
FROM
  `{PROJECT_ID}.shopify_ai.shopify_products_dummy`
"""

# Execute the query
query_job = client.query(query)
results = query_job.result()

# Create an in-memory file object to hold the TSV data
output = io.StringIO()

# Create a CSV writer object with TSV formatting
writer = csv.writer(output, delimiter='\t')

# Write header
writer.writerow([field.name for field in results.schema])

# Write data rows
for row in results:
    writer.writerow([str(row[field.name]) for field in results.schema])

# Get the TSV string
tsv_string = output.getvalue()

# Close the in-memory file
output.close()

# Output the TSV string (you can print it or use it further in your code)
print(tsv_string)

# Feed TSV into Gemini 1.5 Pro

In [114]:
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [115]:
model = genai.GenerativeModel('gemini-1.5-pro')

In [116]:
model.count_tokens(tsv_string)

total_tokens: 97872

In [127]:
response = model.generate_content(f'''Which collection was associated with by-shake-with Slippers?

      {tsv_string}
      '''
)

print(response.text)

The collection associated with the product "by-shake-with: Slippers" is **AW**. 



In [125]:
%%bigquery --project=mesop-431518

SELECT
  *
FROM
  `mesop-431518.shopify_ai.shopify_products_dummy`
ORDER BY subtotal_sold_net_refunds DESC
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,product_type,title,status,created_timestamp,collections,count_variants,has_product_image,total_quantity_sold,subtotal_sold,quantity_sold_net_refunds,subtotal_sold_net_refunds,product_total_discount,product_total_tax
0,267800,Slippers,by-shake-with: Slippers,Active,2024-02-04 17:49:17.841910+00:00,AW,4,True,99.0,8732.0,89.0,7752.0,10.0,1497.0
1,454157,Slippers,major-beat-voice: Slippers,Active,2024-01-29 17:42:00.771122+00:00,SS,4,True,94.0,8460.0,85.0,7560.0,19.0,1504.0
2,997989,Slippers,property-like-later: Slippers,Active,2024-01-21 18:29:30.727458+00:00,Collaborations,8,True,97.0,8468.0,87.0,7498.0,19.0,1505.4
3,189603,Trousers,i-behavior-truth: Trousers,Active,2024-02-16 05:30:17.879526+00:00,SS,3,False,91.0,7781.0,82.0,6926.0,18.0,1167.2
4,458878,Top Hats,enter-likely: Top Hats,Active,2024-07-02 15:00:02.611784+00:00,SS,6,False,93.0,7533.0,84.0,6723.0,19.0,753.4
